In [1]:
from pandas import read_csv, DataFrame, to_numeric, concat
import matplotlib.pylab as plt
from numpy import nan
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,matthews_corrcoef
from yellowbrick.classifier import ConfusionMatrix
from sklearn.tree import DecisionTreeClassifier
from os.path import exists
import scoreValidator

In [2]:
# Checking for existing files
fileAddress ="data_address.txt"
try:
    if exists(fileAddress):
        with open(fileAddress,'r') as text_file:
            Addrs = text_file.readlines()
            Addrs = [addrs.rstrip('\n') for addrs in Addrs]
            Addrs = [addrs.rstrip(' ') for addrs in Addrs]
        text_file.close()
    else:
        print("The file 'data_address.txt' must exists in the main directory. It contains all the address of pre-processed data to reutilize in each model. Please creat it and run this program again.")
        FileNotFoundError("The file 'data_address.txt' must exists in the main directory. It contains all the address of pre-processed data to reutilize in each model. Please creat it and run this program again.")
except:
    print("An error occurred when reading the file: 'data_address.csv'")
    RuntimeError("An error occurred when reading the file: 'data_address.csv'")

In [3]:
# Reading the three pre-processed files for the model
cleaned2 = DataFrame()
significance_filtered = DataFrame()
var_filtered = DataFrame()
try:
    cleaned2 = read_csv(Addrs[0],sep=',')#This is the data without the low variance, significance filter and outliers expulsion
    significance_filtered = read_csv(Addrs[1],sep=',')#This is the model with the significance filter
    var_filtered = read_csv(Addrs[2],sep=',')#This is the model with the low variance filter
except:
    print("Fatal Error: the pre-processed files was not found.")
    FileNotFoundError("Fatal Error: the pre-processed files was not found.")

In [4]:

predictors_cleaned2 = cleaned2.iloc[:,1:]
class_cleaned2 = cleaned2.iloc[:,0]
#-----------------------------------------------
predictors_significance = significance_filtered.iloc[:,1:]
predictors_variance = var_filtered.iloc[:,1:]
#This class can be the same for both predictor_significance and predictor_variance because the number of rows didn't changed on those two filters
class_filtered = var_filtered.iloc[:,0]

In [5]:
#The machine will be trained with the previous years partial data and them whene model works just fine it will be trained with the whole previous years data set and tested with the presente year data set
X_train_cleaned2, X_test_cleaned2, y_train_cleaned2, y_test_cleaned2 = train_test_split(predictors_cleaned2,class_cleaned2,test_size=0.3,random_state=0)
#Splitting significance variables
X_train_significance, X_test_significance, y_train_significance, y_test_significance = train_test_split(predictors_significance,class_filtered,test_size=0.3,random_state=0)
#splitting variance variables
X_train_variance, X_test_variance, y_train_variance, y_test_variance = train_test_split(predictors_variance,class_filtered,test_size=0.3,random_state=0)


In [6]:
#Instantiating an object for each data set
decision_tree_cleaned2 = DecisionTreeClassifier()
decision_tree_significance = DecisionTreeClassifier()
decision_tree_variance = DecisionTreeClassifier()

In [7]:
#Fitting (finally)
decision_tree_cleaned2.fit(X_train_cleaned2,y_train_cleaned2)
decision_tree_significance.fit(X_train_significance,y_train_significance)
decision_tree_variance.fit(X_train_variance,y_train_variance)

DecisionTreeClassifier()

In [8]:
prediction_cleaned2 = decision_tree_cleaned2.predict(X_test_cleaned2)
prediction_significance = decision_tree_significance.predict(X_test_significance)
prediction_variance = decision_tree_variance.predict(X_test_variance)

In [9]:
#Confusion matrix
confusion_matrix_cleaned2 = confusion_matrix(y_test_cleaned2,prediction_cleaned2)
mcc_cleaned2 = matthews_corrcoef(y_test_cleaned2,prediction_cleaned2)
print("MCC Score:", mcc_cleaned2)
print("F2 Score:", scoreValidator.f2_score(confusion_matrix_cleaned2))
print("Weigthed Score:", scoreValidator.weigthedScore(confusion_matrix_cleaned2))
confusion_matrix_cleaned2

MCC Score: 0.5516334740586822
F2 Score: 0.5447470817120622
Weigthed Score: 24.130841121495326


array([[7303,   21],
       [  24,   28]], dtype=int64)

In [11]:
#Confusion matrix Significance
confusion_matrix_significance = confusion_matrix(y_test_significance,prediction_significance)
mcc_significance = matthews_corrcoef(y_test_significance,prediction_significance)
print("MCC Score:", mcc_significance)
print("F2 Score:", scoreValidator.f2_score(confusion_matrix_significance))
print("Weigthed Score:", scoreValidator.weigthedScore(confusion_matrix_significance))
confusion_matrix_significance

MCC Score: 0.4679504872106558
F2 Score: 0.4770992366412214
Weigthed Score: 26.94392523364486


array([[7276,   29],
       [  27,   25]], dtype=int64)

In [13]:
#Confusion matrix Variance
confusion_matrix_variance = confusion_matrix(y_test_variance,prediction_variance)
mcc_variance = matthews_corrcoef(y_test_variance,prediction_variance)
print("MCC Score:", mcc_variance)
print("F2 Score:", scoreValidator.f2_score(confusion_matrix_variance))
print("Weigthed Score:", scoreValidator.weigthedScore(confusion_matrix_variance))
confusion_matrix_variance

MCC Score: 0.42958727053546203
F2 Score: 0.40160642570281124
Weigthed Score: 31.233644859813083


array([[7284,   21],
       [  32,   20]], dtype=int64)

In [34]:
#As espectected the most filtered one is the most precise
#This score is the cost of maintenance when using this ML
print("Score of the model Decision Tree Classifier without dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_cleaned2))
print("----------------------------------------------------------------\n")
print('\n')
print("Score of the model Decision Tree Classifier with Trees Classifier dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_significance))
print("----------------------------------------------------------------\n")
print('\n')
print("Score of the model Decision Tree Classifier with low variance filter dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_variance))
print("----------------------------------------------------------------\n")

Score of the model Decision Tree Classifier without dimensionality reduction:

(The lower the best)


14365.0
----------------------------------------------------------------



Score of the model Decision Tree Classifier with Trees Classifier dimensionality reduction:

(The lower the best)


14405.0
----------------------------------------------------------------



Score of the model Decision Tree Classifier with low variance filter dimensionality reduction:

(The lower the best)


15790.0
----------------------------------------------------------------



In [35]:
#Based on the previous result we choose to use the significance dimensionality reduction for the next step
# Checking for existing files
try:
    present_year_raw = read_csv("Data/air_system_present_year.csv")    
except:
    print("An error occurred when reading the file: 'data_address.csv'")
    RuntimeError("An error occurred when reading the file: 'data_address.csv'")
#Cleaning input sample
X_present_year = DataFrame(present_year_raw[significance_filtered.columns]).iloc[:,1:].replace("na",nan).apply(to_numeric)
y_present_year = DataFrame(present_year_raw[significance_filtered.columns]).iloc[:,0]
new_present_year_raw = concat([y_present_year,X_present_year],axis=1).dropna() 
X_present_year = new_present_year_raw.iloc[:,1:]
y_present_year = new_present_year_raw.iloc[:,0]
decision_tree_significance = DecisionTreeClassifier()
# Training the model with the whole data from previous years
decision_tree_significance.fit(predictors_significance,class_filtered)
prediction = decision_tree_significance.predict(X_present_year)
confusion_matrix_var = confusion_matrix(y_present_year,prediction)
print(confusion_matrix_var)
print("Score of the model Decision Tree Classifier with significance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_var))
print("----------------------------------------------------------------\n")
print("Weigthed score of the model Decision Tree Classifier with significance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.weigthedScore(confusion_matrix_var)/len(y_present_year))
print("----------------------------------------------------------------\n")

[[6492   36]
 [  37   29]]
Score of the model Decision Tree Classifier with significance dimensionality reduction:

(The lower the best)


19585.0
----------------------------------------------------------------

Score rate of the model Decision Tree Classifier with significans dimensionality reduction:

(The lower the best)


2.970124355474674
----------------------------------------------------------------



In [36]:
#Optimizing the model
from accuracyMeassure import accuracyMeassure
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from numpy import ndarray

def accuracyMeassure(y_true:ndarray,y_predict:ndarray)->float:
    confusion_matrix_var = confusion_matrix(y_true,y_predict)
    return scoreValidator.score(confution_matrix_var=confusion_matrix_var)
#Defining a custome score
custom_score = make_scorer(accuracyMeassure)
#Defining parameters to be tested, we let it in blanck because we want it to auto-change them
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'criterion': ['gini', 'entropy']
    }

grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),param_grid=param_grid,cv=5,scoring=custom_score)
grid_searcher.fit(predictors_significance,class_filtered)

print("Best parameters found:\n")
print(grid_searcher.best_params_)
print("\n")
print("Best scores found:\n")
print(grid_searcher.best_score_)
print("\n")

prediction = grid_searcher.best_estimator_.predict(X_present_year)
confusion_matrix_var = confusion_matrix(y_present_year,prediction)
print(confusion_matrix_var)
print("Score of the model Decision Tree Classifier with significance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_var))
print("----------------------------------------------------------------\n")
print("Weigthed score of the model Decision Tree Classifier with significans dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.weigthedScore(confusion_matrix_var)/len(y_present_year))
print("----------------------------------------------------------------\n")

Best parameters found:

{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}


Best scores found:

12665.0


[[6505   23]
 [  41   25]]
Score of the model Decision Tree Classifier with significance dimensionality reduction:

(The lower the best)


21355.0
----------------------------------------------------------------

Score rate of the model Decision Tree Classifier with significans dimensionality reduction:

(The lower the best)


3.2385501971489234
----------------------------------------------------------------



In [37]:
#Based on the previous result we choose to use the significance dimensionality reduction for the next step
# Checking for existing files
try:
    present_year_raw = read_csv("Data/air_system_present_year.csv")    
except:
    print("An error occurred when reading the file: 'data_address.csv'")
    RuntimeError("An error occurred when reading the file: 'data_address.csv'")
#Cleaning input sample
X_present_year = DataFrame(present_year_raw[var_filtered.columns]).iloc[:,1:].replace("na",nan).apply(to_numeric)
y_present_year = DataFrame(present_year_raw[var_filtered.columns]).iloc[:,0]
new_present_year_raw = concat([y_present_year,X_present_year],axis=1).dropna() 
X_present_year = new_present_year_raw.iloc[:,1:]
y_present_year = new_present_year_raw.iloc[:,0]
decision_tree_variance = DecisionTreeClassifier()
# Training the model with the whole data from previous years
decision_tree_variance.fit(predictors_variance,class_filtered)
prediction = decision_tree_variance.predict(X_present_year)
confusion_matrix_var = confusion_matrix(y_present_year,prediction)
print(confusion_matrix_var)
print("Score of the model Decision Tree Classifier with variance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_var))
print("----------------------------------------------------------------\n")
print("Weigthed score of the model Decision Tree Classifier with variance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.weigthedScore(confusion_matrix_var)/len(y_present_year))
print("----------------------------------------------------------------\n")

[[11772    23]
 [   43    36]]
Score of the model Decision Tree Classifier with variance dimensionality reduction:

(The lower the best)


22630.0
----------------------------------------------------------------

Score rate of the model Decision Tree Classifier with variance dimensionality reduction:

(The lower the best)


1.9058447027118073
----------------------------------------------------------------



In [38]:
#Optimizing the best model
#Naive Bayes happens to be the best model based in our accuracy definition, so now is the momento for optimizing the best this model and get the best results
from accuracyMeassure import accuracyMeassure
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from numpy import ndarray

def accuracyMeassure(y_true:ndarray,y_predict:ndarray)->float:
    confusion_matrix_var = confusion_matrix(y_true,y_predict)
    return scoreValidator.score(confution_matrix_var=confusion_matrix_var)
#Defining a custome score
custom_score = make_scorer(accuracyMeassure)
#Defining parameters to be tested, we let it in blanck because we want it to auto-change them
param_grid = {
    'max_depth': [None, 10, 20, 30, 40, 50], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4], 
    'criterion': ['gini', 'entropy']
    }

grid_searcher = GridSearchCV(estimator=DecisionTreeClassifier(),param_grid=param_grid,cv=5,scoring=custom_score)
grid_searcher.fit(predictors_variance,class_filtered)

print("Best parameters found:\n")
print(grid_searcher.best_params_)
print("\n")
print("Best scores found:\n")
print(grid_searcher.best_score_)
print("\n")

prediction = grid_searcher.best_estimator_.predict(X_present_year)
confusion_matrix_var = confusion_matrix(y_present_year,prediction)
print(confusion_matrix_var)
print("Score of the model Decision Tree Classifier with variance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_var))
print("----------------------------------------------------------------\n")
print("Score rate of the model Decision Tree Classifier with variance dimensionality reduction:\n")
print("(The lower the best)")
print("\n")
print(scoreValidator.score(confusion_matrix_var)/len(y_present_year))
print("----------------------------------------------------------------\n")

Best parameters found:

{'criterion': 'entropy', 'max_depth': 50, 'min_samples_leaf': 2, 'min_samples_split': 10}


Best scores found:

12982.0


[[11770    25]
 [   38    41]]
Score of the model Decision Tree Classifier with variance dimensionality reduction:

(The lower the best)


20275.0
----------------------------------------------------------------

Score rate of the model Decision Tree Classifier with variance dimensionality reduction:

(The lower the best)


1.7075122115546573
----------------------------------------------------------------

